In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession \
        .builder \
        .appName("Streaming Word Count") \
        .master("local[3]") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .config("spark.sql.streaming.schemaInference", "true") \
        .getOrCreate()

In [5]:
spark

In [6]:
raw_df = spark.readStream \
    .format("json") \
    .option("path", "d:\\demo\\json") \
    .option("inferSchema",True)\
    .option("maxFilesPerTrigger", 1) \
    .load()

In [7]:
from pyspark.sql.functions import *

In [8]:
explode_df = raw_df.selectExpr("InvoiceNumber", "CreatedTime", "StoreID", "PosID",
                                "CustomerType", "PaymentMethod", "DeliveryType", "DeliveryAddress.City",
                                "DeliveryAddress.State",
                                "DeliveryAddress.PinCode", "explode(InvoiceLineItems) as LineItem")

In [9]:
flattened_df = explode_df \
        .withColumn("ItemCode", expr("LineItem.ItemCode")) \
        .withColumn("ItemDescription", expr("LineItem.ItemDescription")) \
        .withColumn("ItemPrice", expr("LineItem.ItemPrice")) \
        .withColumn("ItemQty", expr("LineItem.ItemQty")) \
        .withColumn("TotalValue", expr("LineItem.TotalValue")) \
        .drop("LineItem")

In [11]:
 invoiceWriterQuery = flattened_df.writeStream \
        .format("json") \
        .queryName("Flattened Invoice Writer") \
        .outputMode("append") \
        .option("path", "D:\\demo\\streamingFolder\\output") \
        .option("checkpointLocation", "D:\\demo\\streamingFolder\\check-point") \
        .trigger(processingTime="1 minute") \
        .start() 

In [ ]:
invoiceWriterQuery.awaitTermination()